# Notes

## Day and short-term strategies

- Scalping: Taking many very small intraday moves, aiming for a few cents or ticks per trade, often with large size and high frequency.
- Momentum trading: Buying strength (or shorting weakness) and “riding the wave” as price and volume accelerate in one direction.
- Range trading: Buying near support and selling near resistance when an asset is oscillating in a well-defined price range.
- Breakout trading: Entering when price pushes through a key support/resistance level or consolidation range, expecting a strong follow-through move.
- News-based trading: Reacting to catalysts like earnings, economic data, or company headlines that cause sudden volatility and volume spikes.[1][3][4][5][7]

## What EMA means in trading terms

- It's a smoothed line on the chart that represents the average price over the last N periods, with recent candles influencing it more than older ones.  
- A “20 EMA” on a 5-minute chart = the exponentially weighted average of the last 20 five-minute closes, updating each bar.  
- Traders use it to:
  - Identify trend direction (price above a rising EMA = uptrend bias, below a falling EMA = downtrend bias).  
  - Act as “dynamic support/resistance” for pullback entries (buy near a rising EMA, sell near a fallin

  So when you see code like `df["ema20"]`, that column is just “the 20-period exponential moving average of price,” used as a faster, more responsive trend line than an SMA.

## Volume Impact

Volume refers to the number of shares or contracts traded for a particular stock during a given period, such as a minute, hour, or day. In day trading, volume is a key indicator showing the strength or interest in a stock's price movement.

### Importance of Volume in Day Trading

- **Liquidity:** Higher volume usually means the stock is easier to buy and sell quickly, making it more suitable for day trading.
- **Confirmation:** Traders often look for high volume to confirm the strength of a price trend or pattern. For example, a breakout above resistance is considered more reliable if it's accompanied by increased volume.
- **Volatility:** Surges in volume can signal heightened interest and potentially greater price swings, which create more opportunities for quick profits—or losses.

### Popular Strategies Incorporating Volume

- Volume Breakouts: Traders buy when a stock breaks above a price resistance on high volume, expecting the momentum to continue.
- Volume Reversals: Sudden spikes in volume after a prolonged trend may signal a reversal or exhaustion, prompting traders to take the opposite position.
- Volume Oscillators: Some strategies use indicators like the Volume Weighted Average Price (VWAP) or On Balance Volume (OBV) to find buy or sell signals based on volume trends.

Volume provides insights into crowd behavior and market dynamics, making it one of the most watched metrics by day traders. By analyzing volume patterns, traders aim to improve timing and confidence in their trades.

## Walk-forward analysis vs back testing

A simulated live trading test on historical data (often called walk-forward analysis, market replay, or bar replay simulation) mimics real-time trading by revealing data gradually as if you're trading live, rather than seeing the full history upfront.​

How it works (vs regular backtest)

| Regular Backtest                          | Simulated Live Trading                                         |
| ----------------------------------------- | -------------------------------------------------------------- |
| Sees all data at once; knows future prices | Data revealed bar-by-bar; can't peek ahead                     |
| Tests full dataset immediately            | Replay mode: play/pause/rewind historical bars at chosen speed |
| Fast but can overfit                      | Slower but realisticdecision-making under time pressure        |

# Code

In [622]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import numpy as np
import time

In [623]:
# sofi = yf.Ticker("SOFI")

In [624]:
# hist = sofi.history(period="8d", interval="1m")
tickers_df = yf.download("SOFI", period="1d", interval="5m").tz_convert("America/New_York")
tickers_df.dropna(inplace=True)
"""
# Example: NYSE calendar (SOFI)
nyse = mcal.get_calendar("NYSE")

# Get all valid trading days over your data range
schedule = nyse.schedule(start_date=hist.index.min().date(),
                         end_date=hist.index.max().date())
trading_days = schedule.index  # DatetimeIndex of trading days
mask = hist.index.tz_convert(None).normalize().isin(trading_days)
print(trading_days)
print(mask)
# Keep only rows whose date is a trading day
hist = hist[mask]
"""

sofi_df = tickers_df.xs("SOFI", level="Ticker", axis=1)
sofi_number_index_df = sofi_df.reset_index()
sofi_df, sofi_number_index_df

/tmp/ipykernel_1894/1445888537.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


(Price                          Close       High        Low       Open  \
 Datetime                                                                
 2025-12-05 09:30:00-05:00  27.320000  27.440001  27.100000  27.365000   
 2025-12-05 09:35:00-05:00  26.980000  27.330000  26.940001  27.309999   
 2025-12-05 09:40:00-05:00  27.445000  27.469999  26.969999  26.985100   
 2025-12-05 09:45:00-05:00  27.309999  27.549999  27.250000  27.445000   
 2025-12-05 09:50:00-05:00  27.235001  27.350000  27.080999  27.309000   
 ...                              ...        ...        ...        ...   
 2025-12-05 15:35:00-05:00  27.764999  27.820000  27.750000  27.795000   
 2025-12-05 15:40:00-05:00  27.655001  27.779900  27.650000  27.770000   
 2025-12-05 15:45:00-05:00  27.677200  27.695000  27.610001  27.655001   
 2025-12-05 15:50:00-05:00  27.754999  27.780001  27.660000  27.670000   
 2025-12-05 15:55:00-05:00  27.780001  27.825001  27.742100  27.745001   
 
 Price                        Volume

In [625]:
fig = go.Figure(data=[go.Candlestick(x=sofi_number_index_df['Datetime'],
                open=sofi_number_index_df['Open'],
                high=sofi_number_index_df['High'],
                low=sofi_number_index_df['Low'],
                close=sofi_number_index_df['Close'])])

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]),                # hide weekends
        dict(bounds=[16, 9.5], pattern="hour"),     # hide 4pm–9:30am

    ]
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    height=800
)
fig.show()

## Strategies

### Momentum long signal: rising EMA + high volume

In [626]:
# Indicators
sofi_ema20_high_volume_df = sofi_df.copy()
sofi_ema20_high_volume_df["ema20"] = sofi_ema20_high_volume_df["Close"].ewm(span=20, adjust=False).mean()
sofi_ema20_high_volume_df["ema20_prev"] = sofi_ema20_high_volume_df["ema20"].shift(1)
sofi_ema20_high_volume_df["ema_rising"] = sofi_ema20_high_volume_df["ema20"] > sofi_ema20_high_volume_df["ema20_prev"]

sofi_ema20_high_volume_df["vol_ma20"] = sofi_ema20_high_volume_df["Volume"].rolling(20).mean()
sofi_ema20_high_volume_df["high_vol"] = sofi_ema20_high_volume_df["Volume"] > sofi_ema20_high_volume_df["vol_ma20"]

# Momentum long signal: price above rising EMA + high volume
sofi_ema20_high_volume_df["long_signal"] = (sofi_ema20_high_volume_df["Close"] > sofi_ema20_high_volume_df["ema20"]) & sofi_ema20_high_volume_df["ema_rising"] & sofi_ema20_high_volume_df["high_vol"]

# Backtest (very simplified, 1 position max, no fees/slippage)
capital = 10_000
position = 0
entry_price = 0.0
pnl = []

for i in range(1, len(sofi_ema20_high_volume_df)):
    t = sofi_ema20_high_volume_df.index[i]
    row = sofi_ema20_high_volume_df.iloc[i]
    prev_row = sofi_ema20_high_volume_df.iloc[i-1]

    # Flatten before close to avoid overnight
    if position != 0 and t.time() >= pd.to_datetime("15:55").time():
        pnl.append((row["Close"] - entry_price) * position)
        position = 0
        entry_price = 0.0
        continue

    # If flat and signal, enter
    if position == 0 and row["long_signal"]:
        entry_price = row["Close"]
        position = capital // entry_price  # all-in sizing
        continue

    # If in position, check TP / SL or exit conditions
    if position != 0:
        # take profit 1%
        if row["High"] >= entry_price * 1.01:
            exit_price = entry_price * 1.01
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0
            continue

        # stop loss 0.5%
        if row["Low"] <= entry_price * 0.995:
            exit_price = entry_price * 0.995
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0
            continue

        # momentum breakdown: close below EMA20
        if row["Close"] < row["ema20"]:
            exit_price = row["Close"]
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0

total_pnl = sum(pnl)
print("Total PnL:", total_pnl)


Total PnL: -46.88901138305664


### Golden Ratio Identifier

In [627]:
def find_swings(df, window=5):
    """Detect swing highs/lows using pivot points"""
    highs = df['High'].rolling(window*2+1, center=True).apply(lambda x: x.iloc[window] if x.iloc[window] == x.max() else np.nan)
    lows = df['Low'].rolling(window*2+1, center=True).apply(lambda x: x.iloc[window] if x.iloc[window] == x.min() else np.nan)
    return highs.dropna(), lows.dropna()

def golden_pocket(df, window=5):
    """Calculate latest Golden Pocket zone"""
    highs, lows = find_swings(df, window)
    
    # Get most recent swing high and low
    recent_high = highs.iloc[-1] if len(highs) > 0 else df['High'].max()
    recent_low = lows.iloc[-1] if len(lows) > 0 else df['Low'].min()
    
    fib_range = abs(recent_high - recent_low)
    
    # Golden Pocket: 61.8% to 65% retracement from high
    gp_upper = recent_high - (0.618 * fib_range)
    gp_lower = recent_high - (0.65 * fib_range)
    
    return {
        'swing_high': recent_high,
        'swing_low': recent_low,
        'gp_upper': gp_upper,
        'gp_lower': gp_lower,
        'in_zone': (gp_lower <= df['Close'].iloc[-1] <= gp_upper)
    }

In [628]:
# Usage
result = golden_pocket(sofi_ema20_high_volume_df)
print(f"Golden Pocket: {result['gp_lower']:.2f} - {result['gp_upper']:.2f}")
print(f"Current price in zone: {result['in_zone']}")

Golden Pocket: 27.68 - 27.69
Current price in zone: False


## Simulated Live Testing with Strategies

In [629]:
day = "2025-12-05"
start_time = pd.Timestamp(f"{day} 09:30", tz="America/New_York")

time_range = pd.date_range(
    start=f"{day} 09:30",
    end=f"{day} 15:55",
    freq="5min",
    tz="America/New_York"   # match your original tz if needed
)

bool_cols = ["EMA_Rising", "High_Vol", "Buy_Signal"]
float_cols = ['Open', 'High', 'Low', 'Close', 'Volume', "EMA20", "Vol_EMA20"]
all_cols = ['Open', 'High', 'Low', 'Close', 'Volume', "EMA20", "EMA_Rising", "Vol_MA20", "High_Vol", "Buy_Signal"]

# Intialize empty dataframe with all columns
empty_df = pd.DataFrame({
            "Open": 0.0,
            "High": 0.0,
            "Low": 0.0,
            "Close": 0.0,
            'Volume': 0.0, 
            "EMA20": 0.0, 
            "EMA_Rising": False,
            "Vol_EMA20": 0.0, 
            "High_Vol": False,
            "Buy_Signal": False,
        }, index=[start_time])

OHLCHistory = empty_df.copy()
OHLCHistory

,Open,High,Low,Close,Volume,EMA20,EMA_Rising,Vol_EMA20,High_Vol,Buy_Signal
2025-12-05 09:30:00-05:00,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,False,False


In [630]:
"""
sofi_ema20_high_volume_df["ema20"] = sofi_ema20_high_volume_df["Close"].ewm(span=20, adjust=False).mean()
sofi_ema20_high_volume_df["ema20_prev"] = sofi_ema20_high_volume_df["ema20"].shift(1)
sofi_ema20_high_volume_df["ema_rising"] = sofi_ema20_high_volume_df["ema20"] > sofi_ema20_high_volume_df["ema20_prev"]

sofi_ema20_high_volume_df["vol_ma20"] = sofi_ema20_high_volume_df["Volume"].rolling(20).mean()
sofi_ema20_high_volume_df["high_vol"] = sofi_ema20_high_volume_df["Volume"] > sofi_ema20_high_volume_df["vol_ma20"]

# Momentum long signal: price above rising EMA + high volume
sofi_ema20_high_volume_df["long_signal"] = (sofi_ema20_high_volume_df["Close"] > sofi_ema20_high_volume_df["ema20"]) & sofi_ema20_high_volume_df["ema_rising"] & sofi_ema20_high_volume_df["high_vol"]
"""

# Functions to calculate EMA20 and Vol_MA20 up to 20, and if EMA_Rising and High_Vol

def calc_ema20(close_col: pd.Series) -> float:
    ema_20 = close_col.ewm(span=20, adjust=False).mean().iloc[-1]

    return ema_20

def is_ema_rising(curr_ema: float, prev_ema: float) -> bool:
    return curr_ema > prev_ema

def calc_vol_ma20(vol_col: pd.Series) -> float:
    return vol_col.mean()

def has_high_vol(curr_vol: float, vol_ma20: float) -> bool:
    return curr_vol > vol_ma20

In [ ]:
# Starting off stats (e.g. captial, position, and profit & losses)
capital = 10_000
position = 0 # max one position
entry_price = 0.0
pnl = []

for i in range(0, 5):
    # Current_price is open price for each 5 min interval
    curr_time = time_range[i]
    current_price = sofi_df["Open"].loc[curr_time]

    # Flatten before close to avoid overnight
    if position != 0 and t.time() >= pd.to_datetime("15:55").time():
        pnl.append((current_price - entry_price) * position)
        position = 0
        entry_price = 0.0
        continue
    
    # Check data and make a decision to buy or not
    if position == 0 and OHLCHistory["Buy_Signal"].iloc[i] == True:
        entry_price = current_price
        position = capital // entry_price  # all-in sizing
        continue
        
    # Wait for close and update data (usually in 5 mins intervals)
    # time.sleep(0.1)

    # Updated OHLC
    OHLCHistory.loc[curr_time] = sofi_df.loc[curr_time]

    OHLCHistory["EMA20"] = OHLCHistory["Close"].ewm(span=20, adjust=False).mean()
    OHLCHistory["Vol_EMA20"] = OHLCHistory["Volume"].ewm(span=20, adjust=False).mean()
    OHLCHistory["EMA_Rising"] = OHLCHistory["EMA20"] > OHLCHistory["EMA20"].shift(1)
    OHLCHistory["High_Vol"] = OHLCHistory["Volume"] > OHLCHistory["Vol_EMA20"] * 1.5
    """
    curr_vol = OHLCHistory["Volume"].iloc[-1]
    if i >= 19: 
        OHLCHistory.loc[OHLCHistory.index[i], "EMA20"] = calc_ema20(OHLCHistory["Close"].iloc[i-19:i+1])

        vol_ma20 = calc_vol_ma20(OHLCHistory["Volume"].iloc[i-19:i+1])

        OHLCHistory.loc[OHLCHistory.index[i], "High_Vol"] = has_high_vol(curr_vol, vol_ma20)
        
    else:
        OHLCHistory.loc[OHLCHistory.index[i], "EMA20"] = calc_ema20(OHLCHistory["Close"].iloc[:i+1])

        vol_ma20 = calc_vol_ma20(OHLCHistory["Volume"].iloc[:i+1])

        OHLCHistory.loc[OHLCHistory.index[i], "High_Vol"] = has_high_vol(curr_vol, vol_ma20)

    OHLCHistory.loc[OHLCHistory.index[i], "Vol_MA20"] = vol_ma20

    if i > 0:
        curr_ema = OHLCHistory["EMA20"].iloc[i] 
        prev_ema = OHLCHistory["EMA20"].iloc[i-1] 
        OHLCHistory.loc[OHLCHistory.index[i], "EMA_Rising"] = is_ema_rising(curr_ema, prev_ema)
    """
    
    OHLCHistory[bool_cols] = OHLCHistory[bool_cols].fillna(False).astype('boolean')
    OHLCHistory[float_cols] = OHLCHistory[float_cols].fillna(0.0).astype('float')

    # After close, analyze data and update data dataframe with new signals

    # Check data and make a decision to sell or not
    if position != 0:
        continue

    # Extend dataframe for next time if under 15:55
    if i < len(time_range) - 1:
        new_time = time_range[i+1]
        template_dict = {
            "Open": 0.0,
            "High": 0.0,
            "Low": 0.0,
            "Close": 0.0,
            'Volume': 0.0, 
            "EMA20": 0.0, 
            "EMA_Rising": False,
            "Vol_EMA20": 0.0, 
            "High_Vol": False,
            "Buy_Signal": False,
        }

        OHLCHistory.loc[new_time] = template_dict
    print(OHLCHistory)

                             Open       High   Low  Close      Volume  EMA20  \
2025-12-05 09:30:00-05:00  27.365  27.440001  27.1  27.32  10458492.0  27.32   
2025-12-05 09:35:00-05:00   0.000   0.000000   0.0   0.00         0.0   0.00   

                           EMA_Rising   Vol_EMA20  High_Vol  Buy_Signal  
2025-12-05 09:30:00-05:00       False  10458492.0     False       False  
2025-12-05 09:35:00-05:00       False         0.0     False       False  
                                Open       High        Low  Close      Volume  \
2025-12-05 09:30:00-05:00  27.365000  27.440001  27.100000  27.32  10458492.0   
2025-12-05 09:35:00-05:00  27.309999  27.330000  26.940001  26.98   3551675.0   
2025-12-05 09:40:00-05:00   0.000000   0.000000   0.000000   0.00         0.0   

                               EMA20  EMA_Rising     Vol_EMA20  High_Vol  \
2025-12-05 09:30:00-05:00  27.320000       False  1.045849e+07     False   
2025-12-05 09:35:00-05:00  27.287619       False  9.800700e+

/tmp/ipykernel_1894/2783394749.py:29: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_1894/2783394749.py:29: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_1894/2783394749.py:29: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_1894/2783394749.py:59: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in